In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer

In [ ]:
from ML import x_train

In [ ]:
train_url = 'https://raw.githubusercontent.com/akshayraj0674/MachineLearning/refs/heads/main/project-1-me-4127-e-2025-26/train.csv'
test_url = 'https://raw.githubusercontent.com/akshayraj0674/MachineLearning/refs/heads/main/project-1-me-4127-e-2025-26/test.csv'
sample_submission_url = 'https://raw.githubusercontent.com/akshayraj0674/MachineLearning/refs/heads/main/project-1-me-4127-e-2025-26/sample_submission.csv'

In [ ]:
train = pd.read_csv(train_url)
test = pd.read_csv(test_url)
sample_submission = pd.read_csv(sample_submission_url)

In [ ]:
print(train.head())
print(test.head())
print(sample_submission.head())

In [ ]:
x_train = train.drop('cost', axis=1)
y_train = train['cost']
x_test = test.copy()

In [ ]:
imputer = SimpleImputer(strategy='mean')
x_train_imputed = imputer.fit_transform(x_train)
x_test_imputed = imputer.transform(x_test)

In [ ]:
model = DecisionTreeRegressor(
    max_depth=5,
    min_samples_leaf=10,
    min_samples_split=20,
    random_state=0
)
model.fit(x_train_imputed, y_train)

In [ ]:
predictions = model.predict(x_test_imputed)

In [ ]:
submission = sample_submission.copy()
submission['cost'] = predictions
submission.to_csv('submission.csv', index=False)